# Making baseline 

In [ ]:
import os
import sys
#os.environ["CUDA_VISIBLE_DEVICES"] = sys.argv[1]
import torch
from deepspeech.train import finetune#, train_new, continue_training
from audio.dataset_factory import DatasetFactory
from sklearn.model_selection import KFold
if torch.cuda.is_available():
    print("Success")
    print(torch.cuda.device_count())

## Training baseline models

* The WER and CER for every sentence in the validation set becomes saved in ./Scores/allScores.csv.

In [4]:
# Finetuning DanSpeechPrimary with no augmentations and with all augmentations on clean and noisy training 
# and validation set 
# 

# 15 epochs

from train_gpu import f_train_gpu

augmentation_list=["speed_perturb","room_reverb","volume_perturb","add_wn","shift_perturb","spec_augment"]
#augmentation_list = []

model_name = "model_name"
#root_dir = '../Data/Tue_Noise_test_split/'# <- Noisy dataset
root_dir = '../Data/Tue_test_split/' # <- Clean dataset

augment_parameters = {'speed_perturb'  : [0.9,1.1],
                  'shift_perturb'  : [-50,50],
                  'room_reverb'    : [0, 0.4, 3.0, 0, 5, 3.0, 0, 5, 3.0, 0, 5, 0.5, 0.5, 0.5],# <- original
                  'volume_perturb' : [5,30],
                  'add_wn'         : [0.5,1.8],
                  'tempo_perturb'  : [-0.25,0.25],
                  'spec_augment'   : [80, 27, 50, 1, 1] # <- original
                 }

f_train_gpu(model_name = model_name, k_folds = None,epochs=15,
            root_dir=root_dir,augmentation_list=augmentation_list,augment_parameters=augment_parameters)

Loading checkpoint model /home/user/.danspeech/models/DanSpeechPrimary.pth
Loading dataset...
Length of dataset: 136
Loading dataset...
Length of dataset: 64

DeepSpeech(
  (conv): MaskConv(
    (seq_module): Sequential(
      (0): Conv2d(1, 32, kernel_size=(41, 11), stride=(2, 2), padding=(20, 5))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Hardtanh(min_val=0, max_val=20, inplace=True)
      (3): Conv2d(32, 32, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Hardtanh(min_val=0, max_val=20, inplace=True)
      (6): Conv2d(32, 96, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5))
      (7): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): Hardtanh(min_val=0, max_val=20, inplace=True)
    )
  )
  (rnns): Sequential(
    (0): BatchRNN(
      (rnn): GRU(2016, 1200, bidirectional=Tr

UnboundLocalError: local variable 'best_transcript' referenced before assignment

## Testing baseline models

* The WER and CER for every sentence in the test set becomes saved in ./Scores/allScoresTest.csv 

* Results and confidence intervals for baseline results can be seen in CI_for_Baseline Results.ipynb. 

In [ ]:
from test_gpu import test_GPU 

test_GPU(
#         "/home/user/.danspeech/models/DanSpeechPrimary.pth",
         "/home/user/DanSpeech-AdvancedMachineLearning/AML Cleaned/model_save_dir/noisy_all_augs.pth", 
#         "/home/user/DanSpeech-AdvancedMachineLearning/Data/Tue_test_split") 
          "/home/user/DanSpeech-AdvancedMachineLearning/Data/Tue_Noise_test_split")  



# Performing cross validation for forward and backward selection

Script performs first round of either forward or backward selection. 

* Results and confidence interval of forward and backward selection can be seen in Forward_&_Backward_Confidence_Interval.ipynb

In [ ]:
from train_gpu import f_train_gpu

feat_sel_method = "forward" # "forward" or "backward"
k_folds = 3
epochs = 15

root_dir = '../Data/Tue_test_split/'

augment_parameters = {'speed_perturb'  : [0.9,1.1],
                  'shift_perturb'  : [-50,50],
                  'room_reverb'    : [0, 0.4, 3.0, 0, 5, 3.0, 0, 5, 3.0, 0, 5, 0.5, 0.5, 0.5],# <- original
                  'volume_perturb' : [5,30],
                  'add_wn'         : [0.5,1.8],
                  'tempo_perturb'  : [-0.25,0.25],
                  'spec_augment'   : [80, 27, 50, 1, 1] # <- original
                 }


if feat_sel_method == "forward":
    

    augmentation_list=["speed_perturb","tempo_perturb","room_reverb","volume_perturb","add_wn","shift_perturb","spec_augment"]

    model_name = "AblationNoise_Baseline"    
    f_train_gpu(model_name = model_name, k_folds = k_folds,root_dir=root_dir,
                augment_parameters=augment_parameters)

    for i in range(len(augmentation_list)):

        aug_method = []
        aug_method.append(augmentation_list[i])

        model_name = "AblationClean_only_" + augmentation_list[i]
        f_train_gpu(model_name, aug_method, k_folds,root_dir=root_dir,
                    augment_parameters=augment_parameters)


if feat_sel_method == "backward":

    augmentation_list=["speed_perturb","tempo_perturb","room_reverb","volume_perturb","add_wn","shift_perturb","spec_augment"]


    model_name = "AblationClean_Baseline_allAugs"
    f_train_gpu(model_name,augmentation_list,k_folds,root_dir=root_dir,
                augment_parameters=augment_parameters)

    for i in range(len(augmentation_list)):
        aug_temp = augmentation_list.copy()
        aug_temp.pop(i)

        model_name = "AblationClean_no_" + augmentation_list[i]

        f_train_gpu(model_name, aug_temp, k_folds,root_dir=root_dir,
                    augment_parameters=augment_parameters)

Success
1
Loading checkpoint model /home/user/.danspeech/models/DanSpeechPrimary.pth


/home/user/DanSpeech-AdvancedMachineLearning/AML Cleaned/deepspeech/train.py:103: NoLoggingDirSpecified: You did not specify a directory for logging training process. Training process will not be logged.
  warnings.warn(


Loading dataset...
Length of dataset: 133
Loading dataset...
Length of dataset: 67

DeepSpeech(
  (conv): MaskConv(
    (seq_module): Sequential(
      (0): Conv2d(1, 32, kernel_size=(41, 11), stride=(2, 2), padding=(20, 5))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Hardtanh(min_val=0, max_val=20, inplace=True)
      (3): Conv2d(32, 32, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Hardtanh(min_val=0, max_val=20, inplace=True)
      (6): Conv2d(32, 96, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5))
      (7): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): Hardtanh(min_val=0, max_val=20, inplace=True)
    )
  )
  (rnns): Sequential(
    (0): BatchRNN(
      (rnn): GRU(2016, 1200, bidirectional=True)
    )
    (1): BatchRNN(
      (batch_norm): SequenceWise (
      Batch

100%|██████████| 9/9 [00:12<00:00,  1.34s/it]


Validation Summary Epoch: [1]	Average WER 34.752	Average CER 9.248	
Found better validated model, saving to /home/user/DanSpeech-AdvancedMachineLearning/AML Cleaned/model_save_dir/AblationNoise_Baseline.pth
started training epoch 2
Successfully exited training and stopped all processes.
Saving all scores into csv...
Loading checkpoint model /home/user/.danspeech/models/DanSpeechPrimary.pth
Loading dataset...
Length of dataset: 133
Loading dataset...
Length of dataset: 67

DeepSpeech(
  (conv): MaskConv(
    (seq_module): Sequential(
      (0): Conv2d(1, 32, kernel_size=(41, 11), stride=(2, 2), padding=(20, 5))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Hardtanh(min_val=0, max_val=20, inplace=True)
      (3): Conv2d(32, 32, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Hardtanh(min_val=0, max_val=20, inplace=True)
      

...

Training Summary Epoch: [9]	Time taken (s): 54	Average Loss 0.195	


100%|██████████| 9/9 [00:10<00:00,  1.20s/it]


Validation Summary Epoch: [9]	Average WER 21.773	Average CER 6.846	
Found better validated model, saving to /home/user/DanSpeech-AdvancedMachineLearning/AML Cleaned/model_save_dir/AblationNoise_Baseline.pth
started training epoch 10
Training Summary Epoch: [10]	Time taken (s): 54	Average Loss 0.087	


100%|██████████| 9/9 [00:10<00:00,  1.20s/it]


Validation Summary Epoch: [10]	Average WER 21.487	Average CER 6.829	
Found better validated model, saving to /home/user/DanSpeech-AdvancedMachineLearning/AML Cleaned/model_save_dir/AblationNoise_Baseline.pth


# Testing of models on test set

In [ ]:
from test_gpu import test_GPU 

test_GPU("/home/user/DanSpeech-AdvancedMachineLearning/AML Cleaned/model_save_dir/noisy_all_augs.pth", 
#         "/home/user/DanSpeech-AdvancedMachineLearning/Data/Tue_test_split") 
          "/home/user/DanSpeech-AdvancedMachineLearning/Data/Tue_Noise_test_split") 



# Hyper-parameter tuning

* Script for hyper-parameter tuning is called train_gpu_hyperparameter_tuning.py

# Final models

In [1]:
from train_gpu import f_train_gpu


model_name = "Clean_original_hyper-parameters.pth"

#root_dir = '../Data/Tue_Noise_test_split/'# <- Noisy dataset
root_dir = '../Data/Tue_test_split/' # <- Clean dataset


#augmentation_list=["room_reverb","spec_augment"] # <- Candidate 1: policy for noisy
#augmentation_list = ["room_reverb"] # <- Candidate 2: policy for noisy
#augmentation_list = ["spec_augment"] # <- Candidate 3: policy for noisy
augmentation_list = ["room_reverb"] # <- policy for clean


augment_parameters = {'speed_perturb'  : [0.9,1.1],
                  'shift_perturb'  : [-50,50],
                  'room_reverb'    : [0, 0.4, 3.0, 0, 5, 3.0, 0, 5, 3.0, 0, 5, 0.5, 0.5, 0.5],# <- original
#                  'room_reverb'    : [0.13, 0.36, 3.19, 0, 5, 7.49, 0, 5, 7.08, 0, 0.5, 0.5, 0.5, 0.5], # <- best for noisy
#                  'room_reverb'    : [0.1, 0.52, 5.9, 0, 5, 3.4, 0, 5, 5.92, 0, 0.5, 0.5, 0.5, 0.5], # <- best for clean
                  'volume_perturb' : [5,30],
                  'add_wn'         : [0.5,1.8],
                  'tempo_perturb'  : [-0.25,0.25],
                  'spec_augment'   : [80, 27, 50, 1, 1] # <- original
#                  'spec_augment'   : [61.2, 31.5, 48.47, 1, 1] # <- best for noisy
                 }


# Out comment everything if the original DanSpeech parameters are used
#augment_prob_dir = {
#                    'room_reverb': 0.64,    # <- Probability for clean
#                    'room_reverb': 0.89, # <- Probability for noisy
#                    'spec_augment': 0.05 # <- Probability for noisy
#                   } 

f_train_gpu(model_name = model_name, k_folds = None,epochs=15,
            root_dir=root_dir,
            augmentation_list=augmentation_list,
            augment_parameters=augment_parameters,
            
           )

Success
1
Loading checkpoint model /home/user/.danspeech/models/DanSpeechPrimary.pth


/home/user/DanSpeech-AdvancedMachineLearning/AML Cleaned/deepspeech/train.py:103: NoLoggingDirSpecified: You did not specify a directory for logging training process. Training process will not be logged.
  warnings.warn(


Loading dataset...
Length of dataset: 136
Loading dataset...
Length of dataset: 64

DeepSpeech(
  (conv): MaskConv(
    (seq_module): Sequential(
      (0): Conv2d(1, 32, kernel_size=(41, 11), stride=(2, 2), padding=(20, 5))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Hardtanh(min_val=0, max_val=20, inplace=True)
      (3): Conv2d(32, 32, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Hardtanh(min_val=0, max_val=20, inplace=True)
      (6): Conv2d(32, 96, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5))
      (7): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): Hardtanh(min_val=0, max_val=20, inplace=True)
    )
  )
  (rnns): Sequential(
    (0): BatchRNN(
      (rnn): GRU(2016, 1200, bidirectional=True)
    )
    (1): BatchRNN(
      (batch_norm): SequenceWise (
      Batch

100%|██████████| 8/8 [00:10<00:00,  1.35s/it]


Validation Summary Epoch: [1]	Average WER 28.655	Average CER 8.741	
Found better validated model, saving to /home/user/DanSpeech-AdvancedMachineLearning/AML Cleaned/model_save_dir/Clean_original_hyper-parameters.pth.pth
started training epoch 2
Training Summary Epoch: [2]	Time taken (s): 53	Average Loss 14.273	


...

100%|██████████| 8/8 [00:10<00:00,  1.35s/it]

Validation Summary Epoch: [14]	Average WER 25.629	Average CER 7.346	
started training epoch 15


Training Summary Epoch: [15]	Time taken (s): 53	Average Loss 0.649	


100%|██████████| 8/8 [00:10<00:00,  1.35s/it]

Validation Summary Epoch: [15]	Average WER 25.430	Average CER 7.362	
Scores saved...
Saving all scores into csv...


# Testing of final models

* Confidence intervals for final models can be found at Final_Models_scores.ipynb

In [2]:
import os
from test_gpu import test_GPU

models = [
          "Clean_original_hyper-parameters.pth",
          "Noisy_original_hyper-parameters_both_augs.pth",
          "Noisy_original_hyper-parameters_only_room_reverb.pth",
          "Noisy_original_hyper-parameters_only_spec_augment.pth",
         ]

for m in models: 
    
    model_path = f"/home/user/DanSpeech-AdvancedMachineLearning/AML Cleaned/model_save_dir/{m}"

    test_GPU(model_path = model_path, 
          data_path="/home/user/DanSpeech-AdvancedMachineLearning/Data/Tue_test_split")

    test_GPU(model_path = model_path, 
             data_path ="/home/user/DanSpeech-AdvancedMachineLearning/Data/Tue_Noise_test_split") 



Success
1
Greedy:
Clean_original_hyper-parameters.pth
Loading dataset...
Length of dataset: 48


100%|██████████| 2/2 [00:04<00:00,  2.41s/it]


Test Summary 	Average WER 21.127	Average CER 5.972	
Saving all test scores into csv...


3-gram language model:
Clean_original_hyper-parameters.pth
Loading dataset...
Length of dataset: 48


100%|██████████| 2/2 [00:05<00:00,  2.68s/it]


Test Summary 	Average WER 12.676	Average CER 5.268	
Saving all test scores into csv...


5-gram language model:
Clean_original_hyper-parameters.pth
Loading dataset...
Length of dataset: 48


...

Test Summary 	Average WER 50.850	Average CER 20.290	
Saving all test scores into csv...


3-gram language model:
Noisy_original_hyper-parameters_only_spec_augment.pth
Loading dataset...
Length of dataset: 59


100%|██████████| 2/2 [00:06<00:00,  3.49s/it]


Test Summary 	Average WER 45.467	Average CER 22.132	
Saving all test scores into csv...


5-gram language model:
Noisy_original_hyper-parameters_only_spec_augment.pth
Loading dataset...
Length of dataset: 59


100%|██████████| 2/2 [00:07<00:00,  3.52s/it]

Test Summary 	Average WER 45.609	Average CER 22.058	
Saving all test scores into csv...
